In [ ]:
import pandas as pd
import hashlib


In [2]:
def log_df(df, limit=5):
    print(df.shape)
    return df.head(limit)

# integer columns that have null
int_dtype = {'playlist_position': 'Int64', 'price': 'Int64', 'serial_number':'Int64'}


## fix title on 'Oli.csv so it uses the full video title

In [3]:
my_oli_df = pd.read_csv('../raw_data/Oli.csv', usecols=['id', 'my id', 'name'])
log_df(my_oli_df)

ValueError: Usecols do not match columns, columns expected but not found: ['my id', 'id', 'name']

In [ ]:
oli_yt_df = pd.read_csv('../raw_data/oli_playlist.csv', usecols=['position', 'video_title', 'video_id'])
log_df(oli_yt_df)

In [ ]:
oli_yt_df['temp_title'] = oli_yt_df['video_title'].replace({"^'Oli ": ""}, regex=True)

oli_yt_df.head()

In [ ]:
merge_df = my_oli_df.merge(oli_yt_df, how = 'outer', left_on='name', right_on='temp_title')
merge_df.head()

In [ ]:
merge_df[merge_df['video_id'].isna()].shape

In [ ]:
merge_df.to_csv('../processed_data/draft/oli_fix_title.csv', index=False)

## add unique id for each model

Some Oli ukes use the same model number for different woods, e.g curly Redwood and redwood are labeled as r. I want each model + wood  to have  id

In [ ]:

my_oli_df = pd.read_csv('../raw_data/Oli.csv', dtype=int_dtype)
log_df(my_oli_df)

In [ ]:
my_oli_df['oli_id'] = ''

for i, row in my_oli_df.iterrows():
    if(pd.isna(row['model'])):
        continue

    temp = row['model'] + row['top_wood'] + row['back_wood']
    # if(not pd.isna(row['model_notes'])):
    #     temp = temp + row['model_notes']
    my_oli_df.loc[i, 'oli_id'] = hashlib.md5(temp.encode()).hexdigest()

In [ ]:
log_df(my_oli_df)

In [ ]:
len(my_oli_df['oli_id'].unique())
# 163

In [ ]:
my_oli_df.to_csv('../raw_data/Oli.csv', index=False)

## update video data 'Oli.csv 

In [ ]:
my_oli_df = pd.read_csv('../raw_data/Oli.csv')
log_df(my_oli_df)

In [ ]:
cols = ['position','video_title','video_id','playlist_id']

oli_yt_df = pd.read_csv('../raw_data/oli_playlist.csv', usecols=cols)
log_df(oli_yt_df)

In [ ]:
new_oli_df = my_oli_df[pd.isna(my_oli_df['video_title']) &  pd.notna(my_oli_df['video_id']) ]
log_df(new_oli_df)

In [ ]:
ids = new_oli_df['video_id'].unique()
ids

In [ ]:
for i, row in oli_yt_df.iterrows():
    video_id = row['video_id']
    if(video_id not in ids):
        continue

    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'playlist_position'] = row['position']
    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'video_title'] = row['video_title']
    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'playlist_id'] = row['playlist_id']
    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'video_type'] = 'sound_sample'
    new_oli_df.loc[new_oli_df['video_id'] == video_id, 'video_provider'] = 'youtube'


In [ ]:
new_oli_df = new_oli_df.astype({"playlist_position": int})

new_oli_df